In [11]:
import sklearn 
from sklearn import linear_model

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
import pandas as pd 
import numpy as np
import random
from statsmodels.tsa.arima_model import ARMA
import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, log_loss


In [2]:
full_df = pd.read_pickle("../ncaa_data/full_df.p")

# Filter Down to Rows Where Each Team Has Actually Played a Previous Game So We Have Data

In [3]:
populated_df = full_df[(full_df["FGM"] > 0) & (full_df["FGM2"] > 0)]

In [4]:
y = populated_df["Team1Score"] - populated_df["Team2Score"]

In [5]:
exclude = ["Team1Score","Team2Score", "date","DayZero","Score","DayNum","Season","WScore","LScore","Team1","Team2","DayZero2","Score2","DayNum2","Season2","date2"]
feature_columns = populated_df.columns
feature_columns = [item for item in feature_columns if item not in exclude]

# Split It Up and Train a Model

In [6]:
# For a random split
# x_train, x_test, y_train, y_test = train_test_split(populated_df[feature_columns], y, test_size=0.1)

In [25]:
#For a split with only the last tournament as test data
train_df = populated_df[(populated_df["DayNum"] < 134) | (populated_df["Season"] != 2017)]
test_df = populated_df[(populated_df["DayNum"] >= 134) & (populated_df["Season"] == 2017)]

x_train = train_df[feature_columns]
x_test = test_df[feature_columns]
y_train = train_df["Team1Score"] - train_df["Team2Score"]
y_test = test_df["Team1Score"] - test_df["Team2Score"]


In [26]:
[print (item) for item in list(zip(feature_columns,models[0].coef_))]

AttributeError: 'tuple' object has no attribute 'coef_'

In [27]:
tourney_matchups = pd.read_pickle("../ncaa_data/tourney_matchups.p")

In [24]:
models = [
            (RandomForestRegressor(), {}),
            (linear_model.Ridge(), {}),
    (linear_model.Lasso(), {}),
    (linear_model.SGDRegressor(), {}),
    (linear_model.BayesianRidge(), {})
         ]

for model_tuple in models:
    model = model_tuple[0]
    print(model)
    model.fit(x_train.fillna(0), y_train)
    predicted = model.predict(x_test)
    predicted[predicted > 0] = .9
    predicted[predicted < 0] = .1
    
    print(log_loss(y_test,predicted))

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
1.05841497796
Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)
1.23830443835
Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)
1.20397280433
SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=None, n_iter=None, penalty='l2',
       power_t=0.25, random_state=None, shuffle=

/home/cody/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


1.23830443835
BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
       fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
       normalize=False, tol=0.001, verbose=False)
1.23830443835


In [18]:
print(y_test, predicted)

77550     7.0
77551     1.0
77552    -4.0
77553    -4.0
77554    18.0
77555    12.0
77556   -15.0
77557    -6.0
77558    20.0
77559    11.0
77560    -9.0
77561     2.0
77562     2.0
77563    10.0
77564     8.0
77565   -20.0
77566    -5.0
77567    -6.0
77568   -10.0
77569   -11.0
77570     6.0
77571    18.0
77572    14.0
77573    22.0
77574    38.0
77575     9.0
77576   -15.0
77577     1.0
77578   -20.0
77579    39.0
         ... 
77584    -1.0
77585    -6.0
77586     9.0
77587     9.0
77588    26.0
77589     6.0
77590    -4.0
77591   -12.0
77592    -3.0
77593   -25.0
77594     4.0
77595    20.0
77596     3.0
77597    -4.0
77598    -7.0
77599     3.0
77600    -7.0
77601   -12.0
77602     3.0
77603    32.0
77604    -1.0
77605    -2.0
77606     1.0
77607    11.0
77608   -12.0
77609   -20.0
77610    24.0
77611   -14.0
77612    -2.0
77613    -7.0
Length: 64, dtype: float64 [ -3.7   6.7  -2.7  -2.6  13.1  10.5  -8.9 -18.3  11.7   6.2   2.6   4.3
   3.5   5.6   4.8 -28.5  -5.9  -7.2 -14.5  -2

In [471]:
predicted

array([ 0.1,  0.1,  0.1,  0.1,  0.9,  0.9,  0.1,  0.1,  0.9,  0.9,  0.9,
        0.1,  0.9,  0.9,  0.9,  0.1,  0.1,  0.1,  0.1,  0.9,  0.9,  0.9,
        0.9,  0.9,  0.9,  0.9,  0.1,  0.9,  0.9,  0.9,  0.1,  0.9,  0.1,
        0.1,  0.9,  0.9,  0.9,  0.9,  0.9,  0.9,  0.1,  0.1,  0.9,  0.9,
        0.9,  0.9,  0.9,  0.9,  0.1,  0.9,  0.9,  0.1,  0.9,  0.9,  0.1,
        0.1,  0.1,  0.9,  0.1,  0.9,  0.9,  0.9,  0.9,  0.9])

In [448]:
models = []
for i in range(0,10):
    print(rand)
    rand = random.randint(0,100)
    linear_model = RandomForestRegressor(random_state=rand)
    linear_model.fit(x_train[feature_columns].fillna(0), y_train)
    
    models.append(linear_model)
    print(i)

96
0
25
1
48
2
36
3
28
4
65
5
3
6
47
7
41
8
62
9


In [462]:
diffs = []
for model in models:
    point_diffs = model.predict(x_test.fillna(0))
    diffs.append(point_diffs)
binary_diffs = np.array(diffs.copy())
binary_diffs[binary_diffs > 0] = 1
binary_diffs[binary_diffs < 0] = 0 
binary_diffs = np.sum(binary_diffs, axis=0)
predicted = binary_diffs * .1
predicted[predicted == 0]=0.1
predicted[predicted == 1]=0.9

In [463]:
list(zip(y_test, predicted))

[(1.0, 0.10000000000000001),
 (1.0, 0.20000000000000001),
 (0.0, 0.5),
 (0.0, 0.60000000000000009),
 (1.0, 0.90000000000000002),
 (1.0, 0.90000000000000002),
 (0.0, 0.10000000000000001),
 (0.0, 0.10000000000000001),
 (1.0, 0.90000000000000002),
 (1.0, 0.90000000000000002),
 (0.0, 0.60000000000000009),
 (1.0, 0.90000000000000002),
 (1.0, 0.90000000000000002),
 (1.0, 0.80000000000000004),
 (1.0, 0.90000000000000002),
 (0.0, 0.10000000000000001),
 (0.0, 0.10000000000000001),
 (0.0, 0.10000000000000001),
 (0.0, 0.10000000000000001),
 (0.0, 0.80000000000000004),
 (1.0, 0.80000000000000004),
 (1.0, 0.90000000000000002),
 (1.0, 0.90000000000000002),
 (1.0, 0.90000000000000002),
 (1.0, 0.90000000000000002),
 (1.0, 0.90000000000000002),
 (0.0, 0.10000000000000001),
 (1.0, 0.90000000000000002),
 (0.0, 0.90000000000000002),
 (1.0, 0.90000000000000002),
 (0.0, 0.10000000000000001),
 (0.0, 0.90000000000000002),
 (0.0, 0.20000000000000001),
 (0.0, 0.10000000000000001),
 (0.0, 0.90000000000000002),
 

In [464]:
log_loss(y_test,predicted)

0.68161007322720124

In [465]:
populated_df["Team1Score"] - populated_df["Team2Score"]

20      -44.0
31        1.0
211      47.0
217      19.0
258       5.0
265      -1.0
295       4.0
310      -8.0
322     -12.0
328      -4.0
344      10.0
358       3.0
378      -6.0
408     -35.0
416      -3.0
422      -2.0
446     -21.0
477     -19.0
503     -21.0
543       6.0
566     -15.0
567      20.0
571      -4.0
610      10.0
645      -7.0
663      12.0
665     -10.0
668      11.0
675      18.0
718     -12.0
         ... 
77584    -1.0
77585    -6.0
77586     9.0
77587     9.0
77588    26.0
77589     6.0
77590    -4.0
77591   -12.0
77592    -3.0
77593   -25.0
77594     4.0
77595    20.0
77596     3.0
77597    -4.0
77598    -7.0
77599     3.0
77600    -7.0
77601   -12.0
77602     3.0
77603    32.0
77604    -1.0
77605    -2.0
77606     1.0
77607    11.0
77608   -12.0
77609   -20.0
77610    24.0
77611   -14.0
77612    -2.0
77613    -7.0
Length: 5162, dtype: float64

In [461]:
diffs = []
for model in models:
    point_diffs = model.predict(tourney_matchups[feature_columns].fillna(0))
    diffs.append(point_diffs)
binary_diffs = np.array(diffs.copy())
binary_diffs[binary_diffs > 0] = 1
binary_diffs[binary_diffs < 0] = 0 
binary_diffs = np.sum(binary_diffs, axis=0)
predicted = binary_diffs * .1
predicted[predicted == 0]=0.1
predicted[predicted == 1]=0.9

In [441]:
y_train

20      -44.0
31        1.0
211      47.0
217      19.0
258       5.0
265      -1.0
295       4.0
310      -8.0
322     -12.0
328      -4.0
344      10.0
358       3.0
378      -6.0
408     -35.0
416      -3.0
422      -2.0
446     -21.0
477     -19.0
503     -21.0
543       6.0
566     -15.0
567      20.0
571      -4.0
610      10.0
645      -7.0
663      12.0
665     -10.0
668      11.0
675      18.0
718     -12.0
         ... 
77424     4.0
77427     6.0
77435     9.0
77436    -6.0
77449   -25.0
77452    13.0
77457    -6.0
77464     2.0
77471    -5.0
77475    11.0
77480     3.0
77483    10.0
77489     4.0
77490    -5.0
77498    -8.0
77499    -4.0
77513   -10.0
77514    -2.0
77516    -1.0
77518     3.0
77519    14.0
77521     6.0
77522     6.0
77525     7.0
77541   -14.0
77542   -28.0
77544   -17.0
77545    15.0
77547     7.0
77548   -15.0
Length: 4226, dtype: float64

In [444]:
log_loss(y_test,predicted)

ValueError: Found input variables with inconsistent numbers of samples: [9112, 936]

In [443]:
# predicted[predicted < 0 ] = 0
# predicted[predicted > 0 ] = 1
# # predicted = (predicted-min(predicted ))/(max(predicted )-min(predicted ))

In [408]:
csv_file = open("submission.csv", "w+")
team_ids = list(tourney_matchups["Team1"])
team_ids2 = list(tourney_matchups["Team2"])
seasons = list(tourney_matchups["Season"])
csv_file.write("ID,Pred\n")
for i in range(len(tourney_matchups["Season"])):
    csv_string = str(seasons[i]) + "_" + str(team_ids[i]) + "_" + str(int(team_ids2[i]))  + "," + str(predicted[i]) + "\n"
    csv_file.write(csv_string)

In [411]:
predicted

array([ 0.3,  0.1,  0.1, ...,  0.7,  0.3,  0.8])

In [57]:
len(tourney_matchups["Season"])

9112

In [78]:
predicted.max()

14.691949517283017

In [168]:
len(full_df.columns)

527